# Extended Context Window Demonstration

This notebook demonstrates the **Extended Context Window** feature for AWS Bedrock's Claude Sonnet 4 model, which supports up to **1 million tokens** in a single request.

## Key Features

- 🚀 **Simple Flag**: Use `enable_extended_context=True` for easy activation
- 🔧 **Manual Control**: Pass custom `additionalModelRequestFields` for fine-grained control
- 📊 **Token Usage**: Monitor input/output token consumption
- 🔄 **Automatic Fallback**: Graceful handling when extended context isn't supported
- 🎯 **Parameter Tracking**: Learn which model/region combinations support which parameters

## ⚠️ Important Notes

- **Beta Feature**: Extended context is a beta service as defined in AWS Service Terms
- **Model Support**: Currently only Claude Sonnet 4 supports 1M token context
- **Pricing**: Separate pricing applies for extended context usage
- **Quotas**: Separate service quotas apply
- **Region Availability**: Not all regions may support this beta feature

## Prerequisites

- AWS credentials configured with Bedrock access
- Access to Claude Sonnet 4 model in your AWS account
- Sufficient quota for extended context usage

## Setup and Imports

In [ ]:
import sys
import json
from pathlib import Path
import logging

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import the LLMManager and related classes
from bestehorn_llmmanager import LLMManager, ParallelLLMManager
from bestehorn_llmmanager.bedrock.models.llm_manager_structures import (
    AuthConfig, RetryConfig, AuthenticationType, RetryStrategy
)
from bestehorn_llmmanager.bedrock.models.model_specific_structures import ModelSpecificConfig
from bestehorn_llmmanager.bedrock.models.parallel_structures import BedrockConverseRequest
from bestehorn_llmmanager.bedrock.tracking.parameter_compatibility_tracker import (
    ParameterCompatibilityTracker
)

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")

## Helper Functions

In [ ]:
def display_response(response, title="Response"):
    """Display a formatted response from LLMManager."""
    print(f"\n{title}")
    print("=" * len(title))
    
    if response.success:
        print(f"✅ Success: {response.success}")
        print(f"🤖 Model: {response.model_used}")
        print(f"🌍 Region: {response.region_used}")
        print(f"⏱️  Total Duration: {response.total_duration_ms:.2f}ms")
        
        # Display content
        content = response.get_content()
        if content:
            print(f"\n💬 Response Content:")
            print("-" * 20)
            # Truncate long responses for readability
            if len(content) > 500:
                print(content[:500] + "...\n[truncated]")
            else:
                print(content)
        
        # Display usage statistics
        input_tokens = response.get_input_tokens()
        output_tokens = response.get_output_tokens()
        total_tokens = response.get_total_tokens()
        
        if total_tokens > 0:
            print(f"\n📊 Token Usage:")
            print(f"   Input Tokens: {input_tokens:,}")
            print(f"   Output Tokens: {output_tokens:,}")
            print(f"   Total Tokens: {total_tokens:,}")
        
        # Display parameter information
        if response.had_parameters_removed():
            print(f"\n⚠️  Parameters Removed: {response.parameters_removed}")
            warnings = response.get_parameter_warnings()
            if warnings:
                print(f"   Warnings:")
                for warning in warnings:
                    print(f"      - {warning}")
    else:
        print(f"❌ Success: {response.success}")
        print(f"🔄 Attempts: {len(response.attempts)}")
    
    if response.warnings:
        print(f"\n⚠️  Warnings:")
        for warning in response.warnings:
            print(f"   - {warning}")

def create_large_text(size_kb: int = 100) -> str:
    """Create a large text string for testing extended context."""
    # Approximate: 1 token ≈ 4 characters
    # 1 KB ≈ 250 tokens
    base_text = "This is a sample text for testing extended context windows. "
    repetitions = (size_kb * 1024) // len(base_text)
    return base_text * repetitions

print("✅ Helper functions defined!")

## Example 1: Simple Extended Context with Flag 🚀

The easiest way to enable extended context is using the `enable_extended_context=True` flag.
This automatically adds the required beta header for Claude Sonnet 4.

In [ ]:
print("🚀 Example 1: Simple Extended Context with Flag")
print("=" * 50)

# Initialize manager with Claude Sonnet 4
manager = LLMManager(
    models=["Claude Sonnet 4"],
    regions=["us-east-1"],
    auth_config=AuthConfig(
        auth_type=AuthenticationType.PROFILE,
        profile_name="default"
    )
)

# Create a large text (simulating a large document)
# For demo purposes, we'll use a moderate size
# In production, you could use up to ~1M tokens
large_text = create_large_text(size_kb=50)  # ~12,500 tokens
print(f"📄 Created text of approximately {len(large_text):,} characters")
print(f"   Estimated tokens: ~{len(large_text) // 4:,}")

# Create message
messages = [
    {
        "role": "user",
        "content": [
            {
                "text": f"Please summarize the following text in 2-3 sentences:\n\n{large_text}"
            }
        ]
    }
]

try:
    # Use extended context with simple flag
    print("\n🔄 Sending request with enable_extended_context=True...")
    response = manager.converse(
        messages=messages,
        enable_extended_context=True,
        inference_config={
            "maxTokens": 512,
            "temperature": 0.3
        }
    )
    
    display_response(response, "🚀 Extended Context Response")
    
    # Show that extended context was used
    print("\n✅ Extended context feature was successfully enabled!")
    print("   The model can now handle up to 1 million tokens.")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Type: {type(e).__name__}")
    print("\n💡 Troubleshooting:")
    print("   - Ensure you have access to Claude Sonnet 4")
    print("   - Check that your region supports the extended context beta")
    print("   - Verify your AWS credentials are configured correctly")

## Example 2: Manual additionalModelRequestFields 🔧

For more control, you can manually specify `additionalModelRequestFields`.
This allows you to combine multiple beta features or use other model-specific parameters.

In [ ]:
print("🔧 Example 2: Manual additionalModelRequestFields")
print("=" * 50)

# Create a message
messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Explain the concept of extended context windows in large language models."
            }
        ]
    }
]

try:
    # Pass custom model-specific parameters directly
    print("\n🔄 Sending request with manual additionalModelRequestFields...")
    response = manager.converse(
        messages=messages,
        additional_model_request_fields={
            "anthropic_beta": [
                "context-1m-2025-08-07"  # Extended context beta header
            ]
        },
        inference_config={
            "maxTokens": 500,
            "temperature": 0.5
        }
    )
    
    display_response(response, "🔧 Manual Configuration Response")
    
    print("\n✅ Successfully used manual additionalModelRequestFields!")
    print("   This approach gives you full control over model-specific parameters.")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 3: Token Usage Reporting 📊

Monitor token consumption to understand the cost and performance of extended context requests.

In [ ]:
print("📊 Example 3: Token Usage Reporting")
print("=" * 40)

# Create texts of different sizes
test_sizes = [
    (10, "Small"),
    (50, "Medium"),
    (100, "Large")
]

results = []

for size_kb, label in test_sizes:
    text = create_large_text(size_kb=size_kb)
    estimated_tokens = len(text) // 4
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "text": f"Provide a one-sentence summary of this text:\n\n{text}"
                }
            ]
        }
    ]
    
    try:
        print(f"\n🔄 Testing {label} text (~{estimated_tokens:,} tokens)...")
        response = manager.converse(
            messages=messages,
            enable_extended_context=True,
            inference_config={"maxTokens": 100}
        )
        
        if response.success:
            results.append({
                "label": label,
                "estimated": estimated_tokens,
                "actual_input": response.get_input_tokens(),
                "output": response.get_output_tokens(),
                "total": response.get_total_tokens()
            })
            print(f"   ✅ Input: {response.get_input_tokens():,} tokens")
            print(f"   ✅ Output: {response.get_output_tokens():,} tokens")
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        results.append({
            "label": label,
            "estimated": estimated_tokens,
            "error": str(e)
        })

# Display summary
print("\n📊 Token Usage Summary")
print("=" * 60)
print(f"{'Size':<10} {'Estimated':<12} {'Actual Input':<15} {'Output':<10} {'Total':<10}")
print("-" * 60)
for result in results:
    if 'error' not in result:
        print(f"{result['label']:<10} {result['estimated']:<12,} "
              f"{result['actual_input']:<15,} {result['output']:<10,} "
              f"{result['total']:<10,}")
    else:
        print(f"{result['label']:<10} {result['estimated']:<12,} ERROR")

print("\n💡 Note: Extended context allows up to 1M tokens, but costs scale with usage.")

## Example 4: Handling Parameter Incompatibility 🔄

The system automatically handles cases where extended context isn't supported,
falling back gracefully to standard context.

In [ ]:
print("🔄 Example 4: Handling Parameter Incompatibility")
print("=" * 50)

# Initialize manager with multiple models (some may not support extended context)
multi_model_manager = LLMManager(
    models=["Claude Sonnet 4", "Claude 3 Haiku"],
    regions=["us-east-1", "us-west-2"],
    auth_config=AuthConfig(
        auth_type=AuthenticationType.PROFILE,
        profile_name="default"
    ),
    retry_config=RetryConfig(
        max_retries=3,
        retry_strategy=RetryStrategy.MODEL_FIRST
    )
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "What are the benefits of extended context windows in AI models?"
            }
        ]
    }
]

try:
    print("\n🔄 Sending request with enable_extended_context=True...")
    print("   The system will try Claude Sonnet 4 first, then fall back if needed.")
    
    response = multi_model_manager.converse(
        messages=messages,
        enable_extended_context=True,
        inference_config={"maxTokens": 400}
    )
    
    display_response(response, "🔄 Fallback Handling Response")
    
    # Check if parameters were removed
    if response.had_parameters_removed():
        print("\n⚠️  Parameter Removal Detected:")
        print(f"   Removed parameters: {response.parameters_removed}")
        print(f"   Model used: {response.model_used}")
        print(f"   Region used: {response.region_used}")
        print("\n💡 The system automatically retried without extended context.")
    else:
        print("\n✅ Extended context was supported by the selected model/region!")
        print(f"   Model: {response.model_used}")
        print(f"   Region: {response.region_used}")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 5: ModelSpecificConfig for Reusable Configuration 🎯

Use `ModelSpecificConfig` to create reusable parameter configurations.

In [ ]:
print("🎯 Example 5: ModelSpecificConfig")
print("=" * 35)

# Create reusable configuration
config = ModelSpecificConfig(
    enable_extended_context=True,
    custom_fields={
        # You can add other model-specific parameters here
    }
)

print("📋 Created ModelSpecificConfig:")
print(f"   enable_extended_context: {config.enable_extended_context}")
print(f"   custom_fields: {config.custom_fields}")

# Initialize manager with default config
config_manager = LLMManager(
    models=["Claude Sonnet 4"],
    regions=["us-east-1"],
    auth_config=AuthConfig(
        auth_type=AuthenticationType.PROFILE,
        profile_name="default"
    ),
    model_specific_config=config  # Set as default
)

messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Explain how configuration objects improve code maintainability."
            }
        ]
    }
]

try:
    print("\n🔄 Sending request (config applied automatically)...")
    # All requests use the config by default
    response = config_manager.converse(
        messages=messages,
        inference_config={"maxTokens": 300}
    )
    
    display_response(response, "🎯 Config-Based Response")
    
    print("\n✅ ModelSpecificConfig was applied automatically!")
    print("   This approach is great for consistent parameter usage across requests.")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"   Type: {type(e).__name__}")

## Example 6: Querying Parameter Compatibility 🔍

The system tracks which model/region combinations support which parameters.
You can query this information for optimization and debugging.

In [ ]:
print("🔍 Example 6: Querying Parameter Compatibility")
print("=" * 45)

# Get compatibility tracker
tracker = ParameterCompatibilityTracker.get_instance()

# Get statistics
stats = tracker.get_statistics()

print("\n📊 Parameter Compatibility Statistics:")
print(f"   Total tracked combinations: {stats.get('total_combinations', 0)}")
print(f"   Compatible combinations: {stats.get('compatible_count', 0)}")
print(f"   Incompatible combinations: {stats.get('incompatible_count', 0)}")

# Check specific combination
test_params = {"anthropic_beta": ["context-1m-2025-08-07"]}
is_incompatible = tracker.is_known_incompatible(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region="us-east-1",
    parameters=test_params
)

print(f"\n🔍 Checking specific combination:")
print(f"   Model: Claude Sonnet 4")
print(f"   Region: us-east-1")
print(f"   Parameters: {test_params}")
print(f"   Known incompatible: {'❌ Yes' if is_incompatible else '✅ No'}")

print("\n💡 The tracker learns from each request to optimize future attempts.")
print("   This helps skip known incompatible combinations during retry.")

## Summary and Best Practices 📝

### Key Takeaways

1. **Simple Flag**: Use `enable_extended_context=True` for easy activation
2. **Manual Control**: Use `additionalModelRequestFields` for fine-grained control
3. **Automatic Fallback**: System handles incompatibility gracefully
4. **Token Monitoring**: Always check token usage for cost management
5. **Configuration Objects**: Use `ModelSpecificConfig` for reusable settings

### Best Practices

- ✅ **Monitor token usage** to understand costs
- ✅ **Use multi-model configuration** for automatic fallback
- ✅ **Check response warnings** to detect parameter removal
- ✅ **Test with smaller contexts** before scaling to 1M tokens
- ✅ **Verify region support** for beta features

### Common Pitfalls

- ❌ Assuming all regions support extended context
- ❌ Not monitoring token consumption
- ❌ Ignoring parameter removal warnings
- ❌ Using extended context when standard context is sufficient

### Additional Resources

- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Claude Model Documentation](https://docs.anthropic.com/)
- [LLMManager Documentation](../docs/forLLMConsumption.md)